### Read data

In [0]:
import pyspark.sql.functions as F

In [0]:
import os

data_folder = '/mnt/2024-team1/'

csv_data = 'JanBDRcount_transpose.csv'
raw_path = os.path.join(data_folder, csv_data)

raw_path


In [0]:
df = spark.read.csv(raw_path, header=True)


In [0]:
display(df)

In [0]:
df.printSchema()

### Convert datatype to int

In [0]:
from pyspark.sql.types import IntegerType

df = df.select(F.col("column"), *[F.col(c).cast(IntegerType()) for c in df.columns[1:]])

In [0]:
df.count()

In [0]:
df.printSchema()

### Remove unwanted features

In [0]:
to_remove = ['FID', 'IID', 'PAT', 'MAT']
df_dropped = df.filter(~F.col('column').isin(to_remove))

In [0]:
display(df_dropped)

In [0]:
# # Check whether null value exist in the dataframe
# # remove the following comment to run the code

# contains_nulls = False
# for c in df.columns:
#   if not df.where(F.col(c).isNull()).limit(1).isEmpty():
#     contains_nulls = True
#     break

# print("if contain null: ", contains_nulls)


In [0]:
df_fillna = df_dropped.na.fill(3)

In [0]:
display(df_fillna)

### One hot encoding

In [0]:
# 19 min
from pyspark.ml.feature import VectorAssembler

df_gene = df_fillna.filter(~F.col('column').isin(['SEX', 'PHENOTYPE']))
df_sex = df_fillna.filter('column = "SEX"')
df_target = df_fillna.filter('column = "PHENOTYPE"')
               

In [0]:
# 10 min

def one_hot_encode_sex(c):
    return F.when(df_sex[c] == 1, F.lit([1, 0])).otherwise(
        F.when(df_sex[c] == 2, F.lit([0, 1]))
      ).alias(c)

df_sex_onehot = df_sex.select(
  F.col("column"), *list(map(one_hot_encode_sex, df.columns[1:]))
  )

def one_hot_encode_gene(c):
    return F.when(df_gene[c] == 0, F.lit([1, 0, 0])).otherwise(
            F.when(df_gene[c] == 1, F.lit([0, 1, 0])).otherwise(
                    F.when(df_gene[c] == 2, F.lit([0, 0, 1])).otherwise(F.lit([0, 0, 0]))
                )
          ).alias(c)

df_gene_onehot = df_gene.select(
  F.col("column"), *list(map(one_hot_encode_gene, df.columns[1:]))
  )


In [0]:
display(df_gene_onehot)

In [0]:
from pyspark.sql.functions import arrays_zip, col, explode

cols = df.columns[1:]

# ref: https://stackoverflow.com/questions/41027315/pyspark-split-multiple-array-columns-into-rows
# ref: https://stackoverflow.com/questions/69162207/pyspark-explode-list-creating-column-with-index-in-list

df_sex_exp = (df_sex_onehot
    .withColumn("tmp", arrays_zip(*cols))
    .select(col("column"), F.posexplode_outer("tmp").alias("index", "tmp"))
    .select(col("column"), col("index"), *[col(f"tmp.{c}") for c in cols]))

df_gene_exp = (df_gene_onehot
    .withColumn("tmp", arrays_zip(*cols))
    .select(col("column"), F.posexplode_outer("tmp").alias("index", "tmp"))
    .select(col("column"), col("index"), *[col(f"tmp.{c}") for c in cols]))


In [0]:
display(df_sex_exp)

In [0]:
df_feat = df_sex_exp.union(df_gene_exp)

In [0]:
display(df_feat)

In [0]:
df_target_with_index = df_target.withColumn("index", F.lit(0))

# re-order columns
df_target_with_index = df_target_with_index.select(F.col("column"), F.col("index"), *[F.col(c) for c in df.columns if c not in ["column", "index"]])

In [0]:
df_target_with_index.printSchema()

In [0]:
df_all = df_feat.union(df_target_with_index)

In [0]:
display(df_all)

### Save one-hot encoded data for subsequent process

In [0]:
write_path = data_folder + "JanBDRcount_transpose_onehot.csv"
write_path

In [0]:
df_all.write.csv(write_path, header=True, sep=',', mode="overwrite")

In [0]:
# write schema
import json

df_all.schema.json()

dbutils.fs.put(data_folder + "one_hot_schema.json", df_all.schema.json(), overwrite=True)


In [0]:
df_test = spark.read.csv(write_path, header=True)

In [0]:
display(df_test)

In [0]:
# check target exist in the saved file

df_test.filter(F.col("column") == "PHENOTYPE").show()